<a href="https://colab.research.google.com/github/Gulfem93/ChatbotProject/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import os
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
intents = json.loads(open('/content/drive/MyDrive/Machine Learning projects/Chatbot Project/intents.json').read())

## Preprocessing

In [ ]:
words = []
classes = []
documents = []

ignore_letters = ['?', '!', '.', ',']

In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))

    if intent["tag"] not in classes:
      classes.append(intent["tag"])

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('/content/drive/MyDrive/Machine Learning projects/Chatbot Project/words.pkl', 'wb'))
pickle.dump(classes, open('/content/drive/MyDrive/Machine Learning projects/Chatbot Project/classes.pkl', 'wb'))


In [ ]:
'''training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_partterns = document[0]
  word_partterns = [lemmatizer.lemmatize(word.lower()) for word in word_partterns]
  for word in words:
    bag.append(1) if word in word_partterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])'''

'training = []\noutput_empty = [0] * len(classes)\n\nfor document in documents:\n  bag = []\n  word_partterns = document[0]\n  word_partterns = [lemmatizer.lemmatize(word.lower()) for word in word_partterns]\n  for word in words:\n    bag.append(1) if word in word_partterns else bag.append(0)\n\n  output_row = list(output_empty)\n  output_row[classes.index(document[1])] = 1\n  training.append([bag, output_row])'

In [ ]:
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_partterns = document[0]
  word_partterns = [lemmatizer.lemmatize(word.lower()) for word in word_partterns]
  for word in words:
    bag.append(1) if word in word_partterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

max_len = max(len(row[0]) for row in training)

for row in training:
  row[0] += [0] * (max_len - len(row[0]))

random.shuffle(training)

train_x = np.array([row[0] for row in training])
train_y = np.array([row[1] for row in training])

## Build Neural Network

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]), ), activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = "softmax"))

sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

hist = model.fit(train_x, train_y, epochs = 200, batch_size = 5, verbose = 1)
model.save("/content/drive/MyDrive/Machine Learning projects/Chatbot Project/chatbot_model.h5", hist)

Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0295 - loss: 3.6084
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0951 - loss: 3.4747
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1597 - loss: 3.2730
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2570 - loss: 2.9650
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2982 - loss: 2.6897
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3454 - loss: 2.3378
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4030 - loss: 2.3187
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5538 - loss: 1.8288
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5535 - loss: 1.7476
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5509 - loss: 1.7012
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5616 - loss: 1.5194
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: